In [ ]:
# 020_multi_chain_travel.py (Notebook용)

from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# === 1단계 체인: 여행지 → 대표 명소 추천 ===
step1_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="당신은 여행 전문가입니다. 여행지에 대해 추천 명소를 제안합니다."),
    HumanMessage(content="사용자가 입력한 여행지: {destination}\n대표 명소 1가지를 추천해주세요.")
])

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

parser = StrOutputParser()  # 문자열로 출력

# --- 1단계 체인: 여행지 → 대표 명소 추천 ---
# Prompt for recommending a single, representative attraction
step1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 전문가입니다. 여행지에 대해 추천 명소를 제안합니다."),
    ("user", "사용자가 입력한 여행지: {destination}\n대표 명소 1가지를 추천해주세요.")
])
step1_chain = step1_prompt | llm | parser

# --- 2단계 체인: 명소 → 상세 정보 제공 ---
# Prompt for providing detailed information about the attraction
step2_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 관광 안내 전문가입니다. 명소에 대한 상세 정보를 제공합니다.
    
    Format:
    명소: {attraction}
    역사: [짧고 간결한 역사 설명]
    특징: [주요 특징, 무엇을 볼 수 있는지 설명]
    방문 팁: [방문 시 유용한 팁, 예: 예매, 시간, 복장 등]
    """),
    ("user", "{attraction}")
])
step2_chain = step2_prompt | llm | parser

# --- LCEL 체인 연결 ---
# Use RunnablePassthrough and RunnableLambda to correctly link the two chains.
# This structure ensures the output of the first chain is passed as a key to the second.
full_chain = (
    {"attraction": step1_chain, "destination": RunnablePassthrough()}
    | step2_chain
)

# --- 실행 ---
destination_input = input("여행지를 입력하세요 (예: 로마, 뉴욕, 도쿄): ")

# 1단계 결과: LCEL의 invoke 메서드를 직접 호출하여 첫 번째 체인만 실행
print(f"\n1단계 추천 명소: {step1_chain.invoke({'destination': destination_input})}")

# 2단계 결과: 전체 체인을 실행하여 최종 결과 확인
print("\n2단계 명소 상세 정보:\n")
final_result = full_chain.invoke({"destination": destination_input})
print(final_result)